In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import tarfile
import torchvision.utils
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split

from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from models import *
import torchvision.transforms as transforms
from torch.utils.data.dataloader import DataLoader
import gc
from utils import * 
from torchvision.utils import save_image

In [2]:
# the path of the originalimages and the saved models
data_dir = './input/gender-classification-dataset'
folder_path1 = 'D:/gende recog adv ex/final networks'

In [3]:
transform = transforms.Compose([transforms.Resize((100, 100)), transforms.ToTensor()])

In [ ]:
batch_size = 128

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
device

In [4]:
# the data loader of the test data set.
dataset_val = ImageFolder(data_dir+"/Validation", transform = transform)

test_dl = DataLoader(dataset_val, batch_size*2, num_workers=4, pin_memory=True)
test_loader = DeviceDataLoader(test_dl, device)

In [11]:
# read the provider model
original_model = GenCnnModel()
original_model = to_device(GenCnnModel(), device)
original_model.load_state_dict(torch.load(folder_path1+'model_main_new.pth'))
to_device(original_model, device);

<All keys matched successfully>

In [13]:
# the test data loader to check if the image was classified correctly or miscalssified
new_batch_size = 1
our_test_loader = DataLoader(dataset_val, new_batch_size, num_workers=0, pin_memory=True)

In [ ]:
# get the list of wrongly classified images
final_acc, wrong_examples, labels, logits = test(original_model, device, our_test_loader)

In [ ]:
wrong_labels = [logits[i].max(1, keepdim=True)[1].item() for i in range(len(labels))]

In [ ]:
# add the perturbation to the image and create the counterfactual example.
corrected_examples = np.zeros([len(wrong_examples)])
perturbed_examples = []
new_labels = []
eps_all = []
for i in range(len(wrong_examples)):
    x, y, prediction = wrong_examples[i], labels[i], logits[i]
    eps = 0.0
#     print(i)
    while True:
        perturbed_image = x.clone()
        perturbed_image.requires_grad = True
        output = original_model(perturbed_image)
        loss = F.nll_loss(output, y)
        original_model.zero_grad()
            # Calculate gradients of model in backward pass
        loss.backward()
        img_grad = perturbed_image.grad.data
        perturbed_image = perturbed_image - eps*img_grad
        output = original_model(perturbed_image)
        new_label = output.max(1, keepdim=True)[1]
        if(new_label.item() == y.item()):
            perturbed_examples.append(perturbed_image.squeeze().data.cpu().numpy())
            new_labels.append(new_label)
            eps_all.append(eps)
            corrected_examples[i] = 1
            print("Image {} has been modified with epsilon {}".format(i, eps))
            break
        eps += 0.0005
        if eps > 0.99:
            break

In [ ]:
# declaring the lists with the original image, the corresponding label, and the perturbations.
real_examples = []
real_labels = []
wrong_predictions = []
corrected_idx = np.where(corrected_examples == 1)
for idx in corrected_idx[0]:
    real_examples.append(wrong_examples[idx].squeeze().data.cpu().numpy())
    real_labels.append(labels[idx].item())
    wrong_predictions.append(wrong_labels[idx])

In [22]:
def get_quartlies(samples):
    q1, med, q3 = np.percentile(samples, [25, 50, 75])
    return q3, q3-q1

In [ ]:
diff = []
tau = 15
for i in range(len(eps_all)):
    diff.append((real_examples[i] - perturbed_examples[i])**2)
    q3, iqr = get_quartlies(diff[i])
    idx = np.where(diff[i] < q3+iqr*tau)
    diff[i][idx]*=0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# boosting the important pixels and neglecting the prxels with low importance.
diff2 = diff
for im in range(len(diff)):
    for layer in range(3):
        for color in range(100):
            for pixel in range(100):
                if diff[im][layer][color][pixel] > 0.0:
                    diff2[im][layer][color][pixel] = 1
                                

                

In [ ]:
# the path to save the explanations of the wrongly classified images.
path_wrong = 'D:/gende recog adv ex/output images/Gender original model/wrong_pri'

In [ ]:
# normalizing the perturbations
for i in range(len(perturbed_examples)):
    for j in range(len(perturbed_examples[i])):
        for k in range(len(perturbed_examples[i][j])):
            for d in range(len(perturbed_examples[i][j][k])):
                perturbed_examples[i][j][k][d] = (perturbed_examples[i][j][k][d]-perturbed_examples[i].min())/(perturbed_examples[i].max()-perturbed_examples[i].min())


In [ ]:
# save the original image and the perturbed image 
for i in range(len(wrong_predictions)):
    z = torch.from_numpy(real_examples[i])
    image1 = z.permute(1,2,0)
    image1 = np.float32(image1)
    z = torch.from_numpy(diff[i])
    image3 = z.permute(1,2,0)
    image3 = np.float32(image3+image1)
    for j in range(len(image3)): 
        for k in range(len(image3[j])):
            for d in range(len(image3[j][k])):
                if image3[j][k][d] > 1:
                    image3[j][k][d] = 1

    plt.imsave(path_wrong+'/image_with_exp/'+str(i)+"label is"+str(state[real_labels[i]])+".jpg",image3)

In [14]:
# get the list of correctly classified images
final_acc, wrong_examples, labels, logits = test_images_true_classified(original_model, device, our_test_loader)

C:\Users\012709558\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [15]:
wrong_labels = [logits[i].max(1, keepdim=True)[1].item() for i in range(len(labels))]

In [18]:
len(corrected_examples)

11256

In [19]:
# add the perturbation to the image and create the counterfactual example.
corrected_examples = np.zeros([len(wrong_examples)])
perturbed_examples = []
new_labels = []
eps_all = []
for i in range(len(wrong_examples)):
    x, y, prediction = wrong_examples[i], labels[i], logits[i]
#     print(prediction.argmax())
    if prediction.argmax() == 0:
        y = 1
        y =torch.tensor([y])
    else:
        y = 0
        y =torch.tensor([y])
    y = y.to(device)
    eps = 0.0
#     print(i)
    while True:
        perturbed_image = x.clone()
        perturbed_image.requires_grad = True
        output = original_model(perturbed_image)
        loss = F.nll_loss(output, y)
        original_model.zero_grad()
            # Calculate gradients of model in backward pass
        loss.backward()
        img_grad = perturbed_image.grad.data
        perturbed_image = perturbed_image - eps*img_grad
        output = original_model(perturbed_image)
        new_label = output.max(1, keepdim=True)[1]
        if(new_label.item() == y.item()):
            perturbed_examples.append(perturbed_image.squeeze().data.cpu().numpy())
            new_labels.append(new_label)
            eps_all.append(eps)
            corrected_examples[i] = 1
            print("Image {} has been modified with epsilon {}".format(i, eps))
            break
        eps += 0.05
        if eps > 0.99:
            corrected_examples[i] = 0
#             perturbed_examples.append(x.squeeze().data.cpu().numpy())
#             new_labels.append(y)
#             eps_all.append(eps)
#             corrected_examples[i] = 1
            break

Image 0 has been modified with epsilon 0.35
Image 1 has been modified with epsilon 0.65
Image 2 has been modified with epsilon 0.39999999999999997
Image 4 has been modified with epsilon 0.8000000000000002
Image 5 has been modified with epsilon 0.44999999999999996
Image 6 has been modified with epsilon 0.1
Image 7 has been modified with epsilon 0.25
Image 8 has been modified with epsilon 0.5499999999999999
Image 9 has been modified with epsilon 0.6
Image 10 has been modified with epsilon 0.7500000000000001
Image 11 has been modified with epsilon 0.49999999999999994
Image 12 has been modified with epsilon 0.2
Image 13 has been modified with epsilon 0.65
Image 15 has been modified with epsilon 0.3
Image 16 has been modified with epsilon 0.39999999999999997
Image 17 has been modified with epsilon 0.7000000000000001
Image 18 has been modified with epsilon 0.8500000000000002
Image 20 has been modified with epsilon 0.7500000000000001
Image 21 has been modified with epsilon 0.15000000000000002

Image 178 has been modified with epsilon 0.3
Image 179 has been modified with epsilon 0.35
Image 180 has been modified with epsilon 0.65
Image 182 has been modified with epsilon 0.3
Image 183 has been modified with epsilon 0.7000000000000001
Image 184 has been modified with epsilon 0.65
Image 186 has been modified with epsilon 0.25
Image 187 has been modified with epsilon 0.7500000000000001
Image 188 has been modified with epsilon 0.2
Image 189 has been modified with epsilon 0.5499999999999999
Image 191 has been modified with epsilon 0.35
Image 192 has been modified with epsilon 0.8500000000000002
Image 193 has been modified with epsilon 0.8500000000000002
Image 194 has been modified with epsilon 0.25
Image 195 has been modified with epsilon 0.44999999999999996
Image 196 has been modified with epsilon 0.3
Image 198 has been modified with epsilon 0.8000000000000002
Image 199 has been modified with epsilon 0.49999999999999994
Image 200 has been modified with epsilon 0.5499999999999999
Im

Image 350 has been modified with epsilon 0.1
Image 351 has been modified with epsilon 0.7500000000000001
Image 352 has been modified with epsilon 0.8000000000000002
Image 353 has been modified with epsilon 0.44999999999999996
Image 355 has been modified with epsilon 0.44999999999999996
Image 357 has been modified with epsilon 0.2
Image 358 has been modified with epsilon 0.44999999999999996
Image 359 has been modified with epsilon 0.7000000000000001
Image 360 has been modified with epsilon 0.44999999999999996
Image 361 has been modified with epsilon 0.65
Image 362 has been modified with epsilon 0.9500000000000003
Image 363 has been modified with epsilon 0.7500000000000001
Image 364 has been modified with epsilon 0.1
Image 365 has been modified with epsilon 0.7000000000000001
Image 366 has been modified with epsilon 0.9500000000000003
Image 367 has been modified with epsilon 0.65
Image 368 has been modified with epsilon 0.2
Image 369 has been modified with epsilon 0.65
Image 370 has been

Image 520 has been modified with epsilon 0.39999999999999997
Image 521 has been modified with epsilon 0.65
Image 522 has been modified with epsilon 0.15000000000000002
Image 523 has been modified with epsilon 0.9000000000000002
Image 524 has been modified with epsilon 0.1
Image 525 has been modified with epsilon 0.7000000000000001
Image 527 has been modified with epsilon 0.65
Image 528 has been modified with epsilon 0.7500000000000001
Image 529 has been modified with epsilon 0.44999999999999996
Image 530 has been modified with epsilon 0.6
Image 531 has been modified with epsilon 0.1
Image 532 has been modified with epsilon 0.25
Image 533 has been modified with epsilon 0.25
Image 534 has been modified with epsilon 0.49999999999999994
Image 535 has been modified with epsilon 0.5499999999999999
Image 536 has been modified with epsilon 0.3
Image 537 has been modified with epsilon 0.44999999999999996
Image 538 has been modified with epsilon 0.9500000000000003
Image 539 has been modified wit

Image 695 has been modified with epsilon 0.7500000000000001
Image 696 has been modified with epsilon 0.6
Image 697 has been modified with epsilon 0.6
Image 698 has been modified with epsilon 0.2
Image 699 has been modified with epsilon 0.25
Image 700 has been modified with epsilon 0.65
Image 701 has been modified with epsilon 0.7000000000000001
Image 703 has been modified with epsilon 0.44999999999999996
Image 704 has been modified with epsilon 0.15000000000000002
Image 705 has been modified with epsilon 0.35
Image 707 has been modified with epsilon 0.39999999999999997
Image 708 has been modified with epsilon 0.39999999999999997
Image 709 has been modified with epsilon 0.05
Image 710 has been modified with epsilon 0.9500000000000003
Image 711 has been modified with epsilon 0.65
Image 712 has been modified with epsilon 0.39999999999999997
Image 713 has been modified with epsilon 0.7000000000000001
Image 714 has been modified with epsilon 0.15000000000000002
Image 715 has been modified w

...

Image 10484 has been modified with epsilon 0.3
Image 10486 has been modified with epsilon 0.25
Image 10487 has been modified with epsilon 0.05
Image 10488 has been modified with epsilon 0.5499999999999999
Image 10489 has been modified with epsilon 0.39999999999999997
Image 10490 has been modified with epsilon 0.49999999999999994
Image 10491 has been modified with epsilon 0.2
Image 10492 has been modified with epsilon 0.15000000000000002
Image 10493 has been modified with epsilon 0.7000000000000001
Image 10494 has been modified with epsilon 0.05
Image 10495 has been modified with epsilon 0.2
Image 10496 has been modified with epsilon 0.44999999999999996
Image 10497 has been modified with epsilon 0.15000000000000002
Image 10498 has been modified with epsilon 0.25
Image 10499 has been modified with epsilon 0.35
Image 10500 has been modified with epsilon 0.25
Image 10501 has been modified with epsilon 0.25
Image 10502 has been modified with epsilon 0.25
Image 10503 has been modified with e

Image 10643 has been modified with epsilon 0.6
Image 10645 has been modified with epsilon 0.3
Image 10646 has been modified with epsilon 0.6
Image 10647 has been modified with epsilon 0.44999999999999996
Image 10648 has been modified with epsilon 0.39999999999999997
Image 10649 has been modified with epsilon 0.7000000000000001
Image 10650 has been modified with epsilon 0.3
Image 10651 has been modified with epsilon 0.15000000000000002
Image 10652 has been modified with epsilon 0.05
Image 10653 has been modified with epsilon 0.2
Image 10654 has been modified with epsilon 0.6
Image 10655 has been modified with epsilon 0.2
Image 10656 has been modified with epsilon 0.2
Image 10657 has been modified with epsilon 0.25
Image 10658 has been modified with epsilon 0.2
Image 10659 has been modified with epsilon 0.7500000000000001
Image 10660 has been modified with epsilon 0.25
Image 10661 has been modified with epsilon 0.49999999999999994
Image 10662 has been modified with epsilon 0.800000000000

Image 10803 has been modified with epsilon 0.39999999999999997
Image 10804 has been modified with epsilon 0.05
Image 10805 has been modified with epsilon 0.7500000000000001
Image 10806 has been modified with epsilon 0.2
Image 10807 has been modified with epsilon 0.39999999999999997
Image 10808 has been modified with epsilon 0.2
Image 10809 has been modified with epsilon 0.2
Image 10810 has been modified with epsilon 0.1
Image 10811 has been modified with epsilon 0.05
Image 10812 has been modified with epsilon 0.15000000000000002
Image 10814 has been modified with epsilon 0.39999999999999997
Image 10815 has been modified with epsilon 0.5499999999999999
Image 10816 has been modified with epsilon 0.05
Image 10817 has been modified with epsilon 0.25
Image 10818 has been modified with epsilon 0.15000000000000002
Image 10819 has been modified with epsilon 0.05
Image 10820 has been modified with epsilon 0.3
Image 10821 has been modified with epsilon 0.39999999999999997
Image 10822 has been mo

Image 10965 has been modified with epsilon 0.6
Image 10966 has been modified with epsilon 0.2
Image 10967 has been modified with epsilon 0.35
Image 10968 has been modified with epsilon 0.49999999999999994
Image 10969 has been modified with epsilon 0.39999999999999997
Image 10970 has been modified with epsilon 0.25
Image 10971 has been modified with epsilon 0.2
Image 10974 has been modified with epsilon 0.6
Image 10975 has been modified with epsilon 0.35
Image 10976 has been modified with epsilon 0.05
Image 10977 has been modified with epsilon 0.05
Image 10978 has been modified with epsilon 0.15000000000000002
Image 10979 has been modified with epsilon 0.6
Image 10980 has been modified with epsilon 0.2
Image 10981 has been modified with epsilon 0.25
Image 10982 has been modified with epsilon 0.15000000000000002
Image 10983 has been modified with epsilon 0.05
Image 10984 has been modified with epsilon 0.25
Image 10985 has been modified with epsilon 0.05
Image 10986 has been modified with

Image 11127 has been modified with epsilon 0.8500000000000002
Image 11128 has been modified with epsilon 0.3
Image 11129 has been modified with epsilon 0.35
Image 11130 has been modified with epsilon 0.25
Image 11131 has been modified with epsilon 0.9000000000000002
Image 11132 has been modified with epsilon 0.5499999999999999
Image 11133 has been modified with epsilon 0.49999999999999994
Image 11134 has been modified with epsilon 0.3
Image 11135 has been modified with epsilon 0.25
Image 11136 has been modified with epsilon 0.3
Image 11137 has been modified with epsilon 0.35
Image 11138 has been modified with epsilon 0.35
Image 11139 has been modified with epsilon 0.6
Image 11140 has been modified with epsilon 0.3
Image 11141 has been modified with epsilon 0.49999999999999994
Image 11142 has been modified with epsilon 0.25
Image 11143 has been modified with epsilon 0.7000000000000001
Image 11144 has been modified with epsilon 0.3
Image 11145 has been modified with epsilon 0.2
Image 111

In [20]:
# declaring the lists with the original image, the corresponding label, and the perturbations.
real_examples = []
real_labels = []
wrong_predictions = []
corrected_idx = np.where(corrected_examples == 1)
for idx in corrected_idx[0]:
    real_examples.append(wrong_examples[idx].squeeze().data.cpu().numpy())
    real_labels.append(labels[idx].item())
    wrong_predictions.append(wrong_labels[idx])

In [23]:
diff = []
tau = 15
for i in range(len(eps_all)):
    diff.append((real_examples[i] - perturbed_examples[i])**2)
    q3, iqr = get_quartlies(diff[i])
    idx = np.where(diff[i] < q3+iqr*tau)
    diff[i][idx]*=0

In [24]:
diff2 = diff
for im in range(len(diff)):
    for layer in range(3):
        for color in range(100):
            for pixel in range(100):
                if diff[im][layer][color][pixel] > 0.0:
                    diff2[im][layer][color][pixel] = 1
                                

In [25]:
len(real_examples)

10538

In [26]:
path_wrong = 'D:/gende recog adv ex/output images/Gender original model/wright_pri'

In [27]:
for i in range(len(perturbed_examples)):
    for j in range(len(perturbed_examples[i])):
        for k in range(len(perturbed_examples[i][j])):
            for d in range(len(perturbed_examples[i][j][k])):
                perturbed_examples[i][j][k][d] = (perturbed_examples[i][j][k][d]-perturbed_examples[i].min())/(perturbed_examples[i].max()-perturbed_examples[i].min())


In [28]:
for i in range(len(wrong_predictions)):
    z = torch.from_numpy(real_examples[i])
    image1 = z.permute(1,2,0)
    image1 = np.float32(image1)
    z = torch.from_numpy(diff[i])
    image3 = z.permute(1,2,0)
    image3 = np.float32(image3+image1)
    for j in range(len(image3)):
        for k in range(len(image3[j])):
            for d in range(len(image3[j][k])):
                if image3[j][k][d] > 1:
                    image3[j][k][d] = 1

    plt.imsave(path_wrong+'/image_with_exp/'+str(i)+"label is"+str(state[real_labels[i]])+".jpg",image3)

In [ ]:
# #real_examples, perturbed_examples, diff, real_labels, wrong_predictions
# fig=plt.figure(figsize=(12, 800))
# columns = 3
# rows = len(wrong_predictions)
# c = 1
# for i in range(len(wrong_predictions)):
#     ax = fig.add_subplot(rows, columns, c)
#     ax.set_title('Predicted as {}'.format(wrong_predictions[i]))
#     ax.set_xticklabels([])
#     ax.set_yticklabels([])
#     ax.set_aspect('equal')
#     z =torch.from_numpy(real_examples[i])
#     image = z.permute(1,2,0)
#     plt.imshow(image)
#     ax = fig.add_subplot(rows, columns, c+1)
#     ax.set_title('Corrected image as {}'.format(real_labels[i]))
#     ax.set_xticklabels([])
#     ax.set_yticklabels([])
#     ax.set_aspect('equal')
#     z =torch.from_numpy(perturbed_examples[i])
#     image = z.permute(1,2,0)
#     plt.imshow(image)
#     ax = fig.add_subplot(rows, columns, c+2)
#     ax.set_title('Pixels cause')
#     ax.set_xticklabels([])
#     ax.set_yticklabels([])
#     ax.set_aspect('equal')
#     z =torch.from_numpy(diff[i])
#     image = z.permute(1,2,0)
#     plt.imshow(image, cmap='hot')
#     c = c + 3
# plt.subplots_adjust(wspace=0, hspace=0)
# plt.tight_layout(True)
# plt.show(block=True)